<a href="https://colab.research.google.com/github/YasJanam/NLP_MODELS_2/blob/main/Labratory_TinyNERModel_5/Labratory_TinyNERModel_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytorch_lightning  seqeval evaluate torchcrf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 66.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6da856a618f0204043aec6400b8a209958bee2169f98c769ff42f9bae89fc30b
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [15]:
!pip install torchcrf --force-reinstall

  Using cached TorchCRF-1.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached TorchCRF-1.1.0-py3-none-any.whl (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.0

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score
from datasets import load_dataset, Dataset
from evaluate import load as load_metric
from transformers import AutoTokenizer
import random
#from torchcrf import CRF

In [11]:
dataset = load_dataset("manu/wnut_17")
dataset = dataset.remove_columns(['id'])
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1287
    })
})

#### **DataAugmentation**

In [12]:
trn_data = dataset['train']
vld_data = dataset['validation']
#tst_data = dataset['test']

In [13]:
import random
from datasets import Dataset

class entity_replacement(object):
  def __init__(self,dataset,num=6):
    self.num_samples = num  # از هر جمله موجود چند جمله جدید بسازیم
    self.dataset = dataset

  def get_sample(self,ner_tag):
    if ner_tag == 1 : # corporation
      len_list = len(corporation)
      index =  random.randint(0, len_list - 1)
      samp = corporation[index]
      return samp

    if ner_tag == 3 : # creative_work
      len_list = len(creative_work)
      index = random.randint(0, len_list - 1)
      samp = creative_work[index]
      return samp

    if ner_tag == 5 : # groups
      len_list = len(groups)
      index = random.randint(0, len_list - 1)
      samp = groups[index]
      return samp

    if ner_tag == 7 : # locations
      len_list = len(locations)
      index = random.randint(0, len_list - 1)
      samp = locations[index]
      return samp

    if ner_tag == 9 : # persons
      len_list = len(persons)
      index = random.randint(0, len_list - 1)
      samp = persons[index]
      return samp

    if ner_tag == 11 : # products
      len_list = len(products)
      index = random.randint(0, len_list - 1)
      samp = products[index]
      return samp

    return None # Return None for other tags

  def new_item(self,item):
    count = 0
    while(count < len(item['tokens'])):
      tag = item['ner_tags'][count]
      next_tag = item['ner_tags'][count + 1] if count + 1 < len(item['ner_tags']) else None
      if tag != 0 :
        if next_tag == 0 :
          item['tokens'][count] = self.get_sample(tag) # Corrected: Use self.get_sample
      count += 1
    return item

  def create_new_items(self,item):
    new_items = []
    for i in range(self.num_samples):
      processed_item = self.new_item(item) # Corrected: Use self.new_item
      new_items.append(processed_item)
    return new_items

  def new_dataset(self):
    new_dataset = []
    for item in self.dataset:
      new_items = self.create_new_items(item) # Corrected: Pass item
      new_dataset.extend(new_items)
    dataset = Dataset.from_list(new_dataset)
    return dataset

In [14]:
class pattern_based_generation(object):
  def __init__(self, templates_list,num):
    self.templates_list = templates_list
    self.num_samples = num

  def convert_text_to_item(self,text):
    tokens = text.split()
    ner_tags = [0] * len(tokens)
    count = 0
    while(count < len(tokens)):
      if tokens[count] == '{person}':
        ner_tags[count] = 9
      elif tokens[count] == '{corporation}':
        ner_tags[count] = 1
      elif tokens[count] == '{group}':
        ner_tags[count] = 5
      elif tokens[count] == '{location}':
        ner_tags[count] = 7
      elif tokens[count] == '{product}':
        ner_tags[count] = 11
      elif tokens[count] == '{creative-work}':
        ner_tags[count] = 3
      count += 1
    item = {}
    item['tokens'] = tokens
    item['ner_tags'] = ner_tags
    return item

  def conver_list_items_to_dataset(self):
    data = {'tokens': [], 'ner_tags': []}
    for item in self.templates_list:
      processed_item = self.convert_text_to_item(item)
      data['tokens'].append(processed_item['tokens'])
      data['ner_tags'].append(processed_item['ner_tags'])
    dataset = Dataset.from_dict(data)
    return dataset

  def create_dataset(self):
      dataset_templates = self.conver_list_items_to_dataset()
      ent_rep = entity_replacement(dataset_templates,self.num_samples)
      new_dataset_template = ent_rep.new_dataset()
      return new_dataset_template

In [15]:
def cocat_datasets(dataset_list):
  final_dataset = {'tokens':[],'ner_tags':[]}
  for dataset in dataset_list:
    for item in dataset:
      final_dataset['tokens'].append(item['tokens'])
      final_dataset['ner_tags'].append(item['ner_tags'])
  final_dataset = Dataset.from_dict(final_dataset)
  return final_dataset

In [16]:
def data_augmentation(dataset,ent_rep_num,pat_gen_num1,pat_gen_num2):
  ent_rep = entity_replacement(dataset,ent_rep_num)
  new_dataset = ent_rep.new_dataset()
  pat_gen1 = pattern_based_generation(all_templates,pat_gen_num1)
  temp_dataset1 = pat_gen1.create_dataset()
  pat_gen2 = pattern_based_generation(more_complex_templates,pat_gen_num2)
  temp_dataset2 = pat_gen2.create_dataset()
  dataset_lists = [new_dataset,temp_dataset1,temp_dataset2]
  final_dataset = cocat_datasets(dataset_lists)
  return final_dataset

In [17]:
train_dataset = data_augmentation(trn_data,9,9,12)
val_dataset = data_augmentation(vld_data,5,4,5)

In [18]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 32760
})

#### **preprocess & Dataset-utilities**

In [ ]:
# Dataset classes
id2label ={'0': "O",
            '1': "B-corporation",
            '2': "I-corporation",
            '3': "B-creative-work",
            '4': "I-creative-work",
            '5': "B-group",
            '6': "I-group",
            '7': "B-location",
            '8': "I-location",
            '9': "B-person",
            '10': "I-person",
            '11': "B-product",
            '12': "I-product"}

label2id = {k:v for v,k in id2label.items()}

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size

In [20]:
def Preprocess_Dataset_2(dataset, tokenizer): #, split: str
    processed_data = {'input_ids':[],'attention_mask':[],'labels':[]}
    for example in dataset : #[split]:
        tokens = example['tokens']
        ner_tags = example['ner_tags']

        filtered_tokens = []
        filtered_ner_tags = []
        for token, tag in zip(tokens, ner_tags):
            if token is not None:
                filtered_tokens.append(token)
                filtered_ner_tags.append(tag)

        tokenized_input = tokenizer(
            filtered_tokens, # Pass the filtered list of tokens
            is_split_into_words=True,
            padding=False,
            truncation=False,
            return_offsets_mapping=True
        )

        word_ids = tokenized_input.word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(filtered_ner_tags[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx


        processed_data['input_ids'].append(tokenized_input['input_ids'])
        processed_data['attention_mask'].append(tokenized_input['attention_mask'])
        processed_data['labels'].append(label_ids)

    processed_dataset = Dataset.from_dict(processed_data)
    return processed_dataset

توجه کنید که دیتای تست را افزایش نمیدهیم تا از دیتای تست دیتاست استفاده کنیم و نتایج را روی دیتای اصلی ارزیابی کنیم

In [21]:
train_data = Preprocess_Dataset_2(train_dataset,tokenizer)
validation_data = Preprocess_Dataset_2(val_dataset,tokenizer)
#test_data = Preprocess_Dataset_2(dataset['test'],tokenizer)

In [22]:
class NERDataset(Dataset):
  def __init__(self, data):
    self.dataset = data

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx] # Get the dictionary item
    return {
            "input_ids": torch.tensor(item['input_ids']), # Convert list to tensor
            "attention_mask": torch.tensor(item['attention_mask']), # Convert list to tensor
            "labels": torch.tensor(item['labels']) # Convert list to tensor
    }

**collate_fn**

متدی برای پد کردن داده ها برای اینکه همه هم اندازه باشند

یک بچ از داده ها را میگرد و همه داده های آن بچ را پد میکند

این متد در دیتالودر استفاده میشه

در واقع **هر بار که دیتالودر یک بچ برمیگردونه این متد روی اون بچ اعمال میشه**

In [23]:
def collate_fn(batch):
  input_ids = [torch.tensor(item['input_ids']) if not isinstance(item['input_ids'], torch.Tensor) else item['input_ids'] for item in batch]
  attention_mask = [torch.tensor(item['attention_mask']) if not isinstance(item['attention_mask'], torch.Tensor) else item['attention_mask'] for item in batch]
  label_ids = [torch.tensor(item['labels']) if not isinstance(item['labels'], torch.Tensor) else item['labels'] for item in batch]

  input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
  attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
  label_ids_padded = pad_sequence(label_ids, batch_first=True, padding_value=-100)

  return {
      "input_ids": input_ids_padded,
      "attention_mask": attention_mask_padded,
      "labels": label_ids_padded
     }

#### **Model**

Encoder-Architecture :

Embedding ⟶ LSTM ⟶ Dropout ⟶  LayerNorm

In [31]:
# ===== Simple Encoder =====
class SimpleEncoder(nn.Module):
  def __init__(self, vocab_size= vocab_size, hidden_dim=32):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, hidden_dim)
    self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, bidirectional=True)
    self.lstm_projection = nn.Linear(hidden_dim * 2, hidden_dim)
    self.conv1d = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=2, padding='same') # Ensure padding is 'same' for matching dimensions
    self.dropout = nn.Dropout(0.2)
    self.ln1 = nn.LayerNorm(hidden_dim)
    self.ln2 = nn.LayerNorm(hidden_dim)
    self.layerNorm = nn.LayerNorm(hidden_dim*2)


  def forward(self, input_ids):
    x = self.embedding(input_ids)

    y,_ = self.lstm(x)  # ouput_dim(y) = 64  & input_dim(x) = 32
    y = self.dropout(y)
    # Project the LSTM output before adding
    #y_projected = self.lstm_projection(y)  # reduce output_dim(y) (64 --> 32)

    #x = x+y_projected  # skip-connection
    #x = self.ln1(x)
    x = self.layerNorm(y)

    """
    x = x.permute(0, 2, 1)
    x = self.conv1d(x)
    x = x.permute(0, 2, 1) # Permute back

    x = self.ln2(x)
    """
    return x

In [32]:
# ===== TokenClassifier Model =====
class TokenClassifier(nn.Module):
  def __init__(self, encoder, num_labels):
    super().__init__()
    self.encoder = encoder
    #self.classifier = nn.Linear(encoder.lstm_projection.out_features, num_labels)
    self.classifier = nn.Linear(encoder.lstm.hidden_size*2,num_labels)

  def forward(self, input_ids):
    x = self.encoder(input_ids)
    logits = self.classifier(x)
    return logits

In [33]:
# ===== Lightning Module =====
metric = load_metric("seqeval")

class NERModule(pl.LightningModule):
  def __init__(self, model, lr=1e-3):
    super().__init__()
    self.model = model
    self.loss_fn = nn.CrossEntropyLoss()
    self.lr = lr
    self.train_dataset = train_data #NERDataset(train_data, tokenizer)
    self.validation_dataset = validation_data #NERDataset(validation_data, tokenizer)


  def forward(self, input_ids):
      return self.model(input_ids)

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=3, shuffle=True, collate_fn=collate_fn) # Use collate_fn

  def val_dataloader(self):
      return DataLoader(self.validation_dataset, batch_size=3, collate_fn=collate_fn)


  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    labels = batch['labels']

    logits = self.model(input_ids)
    loss = self.loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
    self.log("train_loss", loss)
    return loss

  def validation_step(self,batch, batch_idx):
    input_ids = batch['input_ids']
    labels = batch['labels']

    logits = self.model(input_ids)
    pred = torch.argmax(logits, dim=-1)

    loss = self.loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
    self.log("val_loss", loss)

    predictions = []
    references = []
    for i in range(pred.shape[0]):
        preds_row = []
        labels_row = []
        for j in range(pred.shape[1]):
            if labels[i][j].item() != -100:
                preds_row.append(id2label[str(pred[i][j].item())])
                labels_row.append(id2label[str(labels[i][j].item())])
        predictions.append(preds_row)
        references.append(labels_row)

    results = metric.compute(predictions=predictions, references=references)

    for key, value in results.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                self.log(f"val_{key}_{sub_key}", sub_value, prog_bar=True)
        else:
            self.log(f"val_{key}", value, prog_bar=True)



  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.lr)

#### **Train & Test**

In [35]:
# ===== Training =====
from pytorch_lightning.callbacks import EarlyStopping # Import EarlyStopping

device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = 13

encoder = SimpleEncoder(vocab_size=vocab_size)
model = TokenClassifier(encoder, num_labels=num_classes)
ner_module = NERModule(model)

early_stop_callback = EarlyStopping(
    monitor='val_overall_f1',  # Metric to monitor
    min_delta=0.00,           # Minimum change to qualify as an improvement
    patience=4,               # Number of epochs with no improvement after which training will be stopped
    verbose=True,             # Log information to the console
    mode='max'                # 'max' mode means we want to maximize the monitored quantity
)


trainer = pl.Trainer(
    max_epochs=15,
    accelerator=device,
    logger=False, # Set to True to use a logger for better tracking (e.g., TensorBoard)
    callbacks=[early_stop_callback] # Pass the early stopping callback
)
trainer.fit(ner_module)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | TokenClassifier  | 998 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
998 K     Trainable params
0         Non-trainable params
998 K     Total params
3.995     Total estimated model params size

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_overall_f1 improved. New best score: 0.429


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_overall_f1 did not improve in the last 4 records. Best score: 0.466. Signaling Trainer to stop.


نتایج ارزیابی بالا:

val_overall_precision=0.487

val_overall_recall=0.465

val_overall_f1=0.463

num_epochs = 15
num_trained_epoch (with early-stopping) = 9

##### Test

In [36]:
import itertools # Import itertools for flattening

def Preprocess_Dataset_3(dataset, split: str, tokenizer):
    processed_data = []
    for example in dataset[split]:
        tokens = example['tokens']
        ner_tags = example['ner_tags']

        # Tokenize the entire sentence
        tokenized_input = tokenizer(
            tokens,
            is_split_into_words=True,
            padding=False,
            truncation=False,
            return_offsets_mapping=True
        )

        word_ids = tokenized_input.word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(ner_tags[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        # Flatten the lists in case of unexpected nested structures
        flat_input_ids = list(itertools.chain.from_iterable([x] if not isinstance(x, list) else x for x in tokenized_input['input_ids']))
        flat_attention_mask = list(itertools.chain.from_iterable([x] if not isinstance(x, list) else x for x in tokenized_input['attention_mask']))
        flat_label_ids = list(itertools.chain.from_iterable([x] if not isinstance(x, list) else x for x in label_ids))


        processed_data.append({
            'input_ids': flat_input_ids,
            'attention_mask': flat_attention_mask,
            'labels': flat_label_ids
        })

    return processed_data

In [37]:
TEST_DATA = Preprocess_Dataset_3(dataset,'test',tokenizer)

In [38]:
# ============ Test ============
test_dataloader = DataLoader(TEST_DATA, batch_size=3, collate_fn=collate_fn) # Pass TEST_DATA directly

# Put the model in evaluation mode
ner_module.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids']
        labels = batch['labels']

        input_ids = input_ids.to(ner_module.device)
        labels = labels.to(ner_module.device)


        logits = ner_module.model(input_ids)  # Explicitly call the model's forward
        pred = torch.argmax(logits, dim=-1)

        for i in range(pred.shape[0]):
            preds_row = []
            labels_row = []
            for j in range(pred.shape[1]):
                if labels[i][j].item() != -100:
                    if pred[i][j].item() < len(id2label):
                         preds_row.append(id2label[str(pred[i][j].item())])
                    else:
                         preds_row.append("O") # Or another appropriate default label
                    labels_row.append(id2label[str(labels[i][j].item())])
            predictions.append(preds_row)
            references.append(labels_row)


# Compute metrics
metric = load_metric("seqeval")
results = metric.compute(predictions=predictions, references=references)

# Convert numpy/torch objects to standard Python types
def convert_to_python_types(obj):
    if isinstance(obj, dict):
        return {k: convert_to_python_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_python_types(item) for item in obj]
    elif isinstance(obj, (int, float, str, bool, type(None))):
        return obj
    elif isinstance(obj, (torch.Tensor)):
        return obj.tolist()
    elif hasattr(obj, 'item'): # Handle numpy scalars
        return obj.item()
    else:
        return str(obj) # Convert other types to string

results_python = convert_to_python_types(results)


import json
print(json.dumps(results_python, indent=4))

{
    "corporation": {
        "precision": 0.07692307692307693,
        "recall": 0.07575757575757576,
        "f1": 0.07633587786259544,
        "number": 66
    },
    "creative-work": {
        "precision": 0.0,
        "recall": 0.0,
        "f1": 0.0,
        "number": 142
    },
    "group": {
        "precision": 0.047058823529411764,
        "recall": 0.024242424242424242,
        "f1": 0.032,
        "number": 165
    },
    "location": {
        "precision": 0.09401709401709402,
        "recall": 0.07333333333333333,
        "f1": 0.08239700374531835,
        "number": 150
    },
    "person": {
        "precision": 0.22905027932960895,
        "recall": 0.09557109557109557,
        "f1": 0.13486842105263158,
        "number": 429
    },
    "product": {
        "precision": 0.02,
        "recall": 0.007874015748031496,
        "f1": 0.011299435028248588,
        "number": 127
    },
    "overall_precision": 0.1154562383612663,
    "overall_recall": 0.057460611677479144,
   

#### **Names & Templates**

In [8]:
products = [
    # محصولات الکترونیکی و تکنولوژی
    "iPhone",
    "iPad",
    "MacBook",
    "Surface",
    "Galaxy",
    "PlayStation",
    "Xbox",
    "Kindle",
    "Pixel",
    "AirPods",

    # نوشیدنی‌ها
    "CocaCola",
    "Pepsi",
    "Fanta",
    "Sprite",
    "RedBull",
    "Monster",
    "Nescafe",
    "Lipton",
    "Evian",
    "Heineken",

    # خوراکی‌ها و تنقلات
    "Oreo",
    "KitKat",
    "Snickers",
    "M&M",
    "Pringles",
    "Nutella",
    "PeanutButter",
    "Lay's",
    "Doritos",
    "Haribo",

    # محصولات مد و پوشاک
    "NikeAir",
    "AdidasUltra",
    "PumaShoes",
    "LevisJeans",
    "RayBan",
    "GucciBag",
    "LouisVuitton",
    "ChanelPerfume",
    "HermesScarf",
    "RolexWatch"
]


In [7]:
persons = [
    # دانشمندان و مخترعان
    "Einstein",
    "Newton",
    "Tesla",
    "Curie",
    "Galileo",
    "DaVinci",
    "Hawking",
    "Turing",
    "Feynman",
    "Edison",

    # هنرمندان و نویسندگان
    "Shakespeare",
    "Hemingway",
    "Poe",
    "Orwell",
    "Picasso",
    "VanGogh",
    "Klimt",
    "Dali",
    "Michelangelo",
    "Raphael",

    # شخصیت‌های تاریخی و سیاسی
    "Gandhi",
    "Lincoln",
    "Churchill",
    "Mandela",
    "Napoleon",
    "Cleopatra",
    "Alexander",
    "Caesar",
    "Washington",
    "KingJr",

    # ورزشکاران و افراد مشهور مدرن
    "Messi",
    "Ronaldo",
    "SerenaWilliams",
    "MichaelJordan",
    "LeBronJames",
    "TigerWoods",
    "RogerFederer",
    "UsainBolt",
    "TomBrady",
    "SimoneBiles",

    # بازیگران و شخصیت‌های فرهنگی
    "Beyonce",
    "Adele",
    "BradPitt",
    "AngelinaJolie",
    "LeonardoDiCaprio",
    "TomHanks",
    "EmmaWatson",
    "ScarlettJohansson",
    "DwayneJohnson",
    "WillSmith"
]


In [6]:
locations = [
    # کشورهای معروف
    "UnitedStates",
    "Canada",
    "Brazil",
    "Germany",
    "France",
    "Italy",
    "Spain",
    "UnitedKingdom",
    "Japan",
    "China",
    "India",
    "Australia",
    "Mexico",
    "Russia",
    "Egypt",
    "SouthAfrica",
    "Argentina",
    "Thailand",
    "Turkey",
    "Netherlands",

    # شهرهای معروف
    "NewYork",
    "LosAngeles",
    "Chicago",
    "London",
    "Paris",
    "Berlin",
    "Rome",
    "Madrid",
    "Tokyo",
    "Beijing",
    "Moscow",
    "Sydney",
    "Toronto",
    "Dubai",
    "Istanbul",

    # مکان‌های دیدنی و معروف
    "EiffelTower",
    "StatueOfLiberty",
    "GreatWall",
    "TajMahal",
    "Colosseum",
    "BigBen",
    "Pyramids",
    "MachuPicchu",
    "ChristTheRedeemer",
    "GoldenGateBridge",
    "Santorini",
    "NiagaraFalls",
    "MountEverest",
    "GrandCanyon",
    "SagradaFamilia"
]


In [5]:
groups = [
    # تیم‌ها و گروه‌های ورزشی
    "Barcelona",
    "RealMadrid",
    "ManchesterUnited",
    "Liverpool",
    "Juventus",
    "BayernMunich",
    "ChicagoBulls",
    "LA Lakers",
    "GoldenStateWarriors",
    "NewYorkYankees",
    "DallasCowboys",
    "AllBlacks",
    "PittsburghSteelers",
    "TorontoMapleLeafs",

    # گروه‌ها و بندهای موسیقی
    "Beatles",
    "Queen",
    "Coldplay",
    "LinkinPark",
    "ImagineDragons",
    "Metallica",
    "RollingStones",
    "Maroon5",
    "OneDirection",
    "ABBA",

    # گروه‌ها و استودیوهای فیلم و تلویزیون
    "Pixar",
    "WarnerBros",
    "MarvelStudios",
    "DCComics",
    "Paramount",
    "Universal",
    "DreamWorks",
    "StudioGhibli",
    "BBCStudios",

    # گروه‌های هنری و فرهنگی
    "Bauhaus",
    "Fluxus",
    "Dada",
    "Impressionists",
    "Surrealists",
    "Renaissance",
    "Postmodernists",
    "AbstractExpressionists",
    "Cubists",
    "PopArtists"
]


In [4]:
corporation = [
    # شرکت‌های فناوری و نرم‌افزاری
    "Google",
    "Microsoft",
    "Apple",
    "Amazon",
    "Facebook",
    "IBM",
    "Oracle",
    "Intel",
    "Adobe",
    "Salesforce",
    "Spotify",
    "Snapchat",
    "Twitter",
    "Tesla",
    "Nvidia",
    "Uber",
    "Airbnb",
    "Dropbox",
    "Slack",
    "Zoom",
    "Atlassian",
    "Shopify",
    "GitHub",
    "RedHat",
    "LinkedIn",
    "PayPal",

    # شرکت‌های تجاری و برندهای شناخته‌شده
    "Nike",
    "Adidas",
    "CocaCola",
    "Pepsi",
    "Starbucks",
    "Disney",
    "Netflix",
    "Sony",
    "Samsung",
    "Toyota",
    "BMW",
    "Intel",
    "Huawei",
    "LG",
    "McDonalds",
    "KFC",
    "Unilever",
    "ProcterGamble",
    "Nestle",
    "Volkswagen",
    "Heineken",
    "Ikea",
    "H&M",
    "Zara",
    "Lego",
    "Patagonia"
]


In [3]:
creative_work = [
    # نقاشی‌ها و مجسمه‌ها
    "StarryNight",
    "MonaLisa",
    "Persistence",
    "Scream","Guernica",
    "PearlEarring","KissKlimt","NightWatch","Venus","Creation","GreatWave","Demoiselles","Athens","LasMeninas","Gothic","EarthlyDelights","Arnolfini","HayWain",
    "Temeraire","Shalott","SonOfMan","KissBrancusi","Thinker","Burghers","KissRodin",

    # فیلم‌ها
    "Inception",
    "Titanic",
    "Avatar",
    "Gladiator",
    "Casablanca",
    "Joker",
    "Amelie",
    "Interstellar",
    "Parasite",
    "Frozen",

    # سریال‌ها
    "Friends",
    "BreakingBad",
    "GameOfThrones",
    "Sherlock",
    "StrangerThings",
    "Westworld",
    "Dexter",
    "Lost",
    "Vikings",

    # کتاب‌ها و داستان‌ها
    "HarryPotter",
    "LordOfTheRings",
    "PrideAndPrejudice",
    "Hobbit",
    "1984",
    "Dune",
    "MobyDick",
    "Gatsby",
    "Dracula",
    "Frankenstein"
]


In [2]:
all_templates = [
    # ===== PERSON =====
    "{person} works at {corporation}.",
    "{person} traveled to {location}.",
    "{person} lives in {location}.",
    "{person} is a member of {group}.",
    "{person} won an award for {creative-work}.",
    "{person} invented {product}.",
    "{person} wrote {creative-work}.",
    "{person} designed {product}.",
    "{person} joined {corporation} as CEO.",
    "{person} is the leader of {group}.",
    "{person} founded {corporation}.",
    "{person} recommended {product}.",
    "{person} directed {creative-work}.",
    "{person} created {product} with {corporation}.",
    "{person} gave a speech in {location}.",
    "{person} performed with {group}.",
    "{person} collaborated with {corporation} on {creative-work}.",
    "{person} was born in {location}.",
    "{person} studied at {location}.",
    "{person} supports {group}.",
    "{person} criticized {corporation}.",
    "{person} published {creative-work} through {corporation}.",
    "{person} announced {product} in {location}.",
    "{person} organized an event with {group}.",
    "{person} appeared in {creative-work}.",
    "{person} worked for {corporation} in {location}.",
    "{person} launched {product} at {corporation}.",
    "{person} reviewed {creative-work}.",
    "{person} collaborated with {person} on {creative-work}.",
    "{person} visited {corporation} in {location}.",

    # ===== CORPORATION =====
    "{corporation} announced a new project.",
    "{corporation} opened a branch in {location}.",
    "{corporation} produces {product}.",
    "{corporation} collaborated with {group} on a campaign.",
    "{corporation} is competing with {corporation}.",
    "{corporation} published {creative-work}.",
    "{corporation} hired {person}.",
    "{corporation} released {creative-work} in {location}.",
    "{corporation} is based in {location}.",
    "{corporation} sponsors {group}.",
    "{corporation} launched {product} last year.",
    "{corporation} invested in {group}.",
    "{corporation} signed a contract with {person}.",
    "{corporation} funded {creative-work}.",
    "{corporation} acquired {corporation}.",
    "{corporation} built an office in {location}.",
    "{corporation} tested {product} with {group}.",
    "{corporation} developed {creative-work} with {person}.",
    "{corporation} presented {product} in {location}.",
    "{corporation} partnered with {corporation} to release {product}.",

    # ===== GROUP =====
    "{group} organized a concert in {location}.",
    "The leader of {group} is {person}.",
    "{group} was founded in 1990.",
    "{group} competed with {group}.",
    "{group} released {creative-work}.",
    "{group} uses {product}.",
    "{group} performed in {location}.",
    "{group} collaborated with {corporation}.",
    "{group} recruited {person}.",
    "{group} protested against {corporation}.",
    "{group} supported {person}.",
    "{group} traveled to {location}.",
    "{group} created {creative-work}.",
    "{group} bought {product}.",
    "{group} presented {creative-work} in {location}.",
    "{group} featured {person} in {creative-work}.",
    "{group} launched {product} with {corporation}.",
    "{group} participated in {creative-work}.",
    "{group} campaigned in {location}.",
    "{group} honored {person} for {creative-work}.",

    # ===== LOCATION =====
    "{location} is a popular tourist spot.",
    "{corporation} opened a new office in {location}.",
    "{group} held a meeting in {location}.",
    "{creative-work} is set in {location}.",
    "{product} is sold in {location}.",
    "{person} gave a talk in {location}.",
    "{corporation} is headquartered in {location}.",
    "{person} traveled to {location} with {group}.",
    "{corporation} launched {product} in {location}.",
    "{group} performed {creative-work} in {location}.",
    "{person} studied at a university in {location}.",
    "{corporation} invested in {location}.",
    "{product} was tested in {location}.",
    "{creative-work} premiered in {location}.",
    "{group} organized a march in {location}.",
    "{person} visited {location} during holidays.",
    "{corporation} built a factory in {location}.",
    "{group} opened an office in {location}.",
    "{creative-work} was filmed in {location}.",
    "{person} was honored in {location}.",

    # ===== PRODUCT =====
    "{product} is manufactured by {corporation}.",
    "Many customers bought {product} in {location}.",
    "{person} recommended {product}.",
    "{product} is more expensive than {product}.",
    "The latest version of {product} was released by {corporation}.",
    "{creative-work} features {product}.",
    "{person} designed {product}.",
    "{group} uses {product} in their work.",
    "{product} is used in {creative-work}.",
    "{corporation} sells {product} in {location}.",
    "{person} bought {product} from {corporation}.",
    "{group} tested {product}.",
    "{product} was released in {location}.",
    "{product} supports {creative-work}.",
    "{person} promoted {product}.",
    "{corporation} donated {product} to {group}.",
    "{product} was reviewed in {creative-work}.",
    "{group} adopted {product}.",
    "{corporation} showcased {product} at {location}.",
    "{person} developed {product} at {corporation}.",

    # ===== CREATIVE-WORK =====
    "{creative-work} was released last year.",
    "{creative-work} was created by {group}.",
    "The director of {creative-work} is {person}.",
    "{creative-work} won an award.",
    "{corporation} published {creative-work}.",
    "{person} reviewed {creative-work}.",
    "{group} performed {creative-work}.",
    "{creative-work} was filmed in {location}.",
    "{creative-work} was inspired by {creative-work}.",
    "{creative-work} stars {person}.",
    "{creative-work} is about {group}.",
    "{creative-work} is sold in {location}.",
    "{creative-work} was banned by {group}.",
    "{creative-work} features {product}.",
    "{corporation} financed {creative-work}.",
    "{person} directed {creative-work}.",
    "{group} produced {creative-work}.",
    "{creative-work} was presented in {location}.",
    "{creative-work} competed with {creative-work}.",
    "{person} won a prize for {creative-work}.",
]

print("Number of templates:", len(all_templates))


Number of templates: 130


In [1]:
more_complex_templates = [
    # ==== Compound with conjunctions ====
    "{person} wrote {creative-work}, and {corporation} decided to publish it in {location}.",
    "{group} performed {creative-work}, but {corporation} refused to finance it.",
    "{person} invented {product}, so {corporation} promoted it worldwide.",
    "{corporation} supported {group}, yet {person} opposed their decision.",
    "{creative-work} won an award, and {person} thanked {group} during the ceremony.",

    # ==== Passive voice ====
    "{product} was designed by {person} and later improved by {corporation}.",
    "{creative-work} was reviewed by {group} before being released in {location}.",
    "{corporation} was founded by {person} in {location}.",
    "{group} was sponsored by {corporation} for their {creative-work}.",
    "{person} was awarded for {creative-work} in {location}.",

    # ==== Relative clauses ====
    "{person}, who previously worked at {corporation}, now leads {group}.",
    "{product}, which was launched by {corporation}, became popular in {location}.",
    "{creative-work}, which was inspired by {product}, was directed by {person}.",
    "{corporation}, whose headquarters is in {location}, released {creative-work}.",
    "{group}, which includes {person}, collaborated with {corporation}.",

    # ==== Time expressions ====
    "Last year, {person} created {product} in partnership with {corporation}.",
    "In 2020, {corporation} launched {creative-work} in {location}.",
    "For many years, {group} has been performing in {location}.",
    "In the future, {corporation} will expand to {location}.",
    "Recently, {person} reviewed {creative-work} for {corporation}.",

    # ==== Conditional variants ====
    "If {corporation} releases {product}, {group} will adopt it in {location}.",
    "As long as {person} supports {group}, {creative-work} will continue.",
    "Even if {product} fails, {corporation} will try again in {location}.",
    "If {creative-work} is successful, {person} will win an award.",
    "Provided that {corporation} invests in {group}, {product} will improve.",

    # ==== Comparisons ====
    "{person} is more influential in {group} than {person} in {corporation}.",
    "{creative-work} gained more attention in {location} than {creative-work}.",
    "{product} is cheaper in {location} than in {location}.",
    "{corporation} hired more employees than {corporation} last year.",
    "{group} produced more {creative-work} than {group}.",

    # ==== Questions ====
    "Who is responsible for {product} at {corporation}?",
    "What inspired {person} to write {creative-work}?",
    "When was {corporation} founded in {location}?",
    "Which {group} organized the event in {location}?",
    "Why did {corporation} acquire {product} from {group}?",
    "How did {person} collaborate with {corporation} on {creative-work}?",
    "Where did {group} perform {creative-work} last year?",
    "Whose {product} was presented by {corporation}?",
    "Which {creative-work} did {person} direct in {location}?",
    "What role did {corporation} play in {creative-work}?",

    # ==== Miscellaneous complex ====
    "After {person} presented {product}, {corporation} decided to invest.",
    "Before {group} released {creative-work}, {person} edited it carefully.",
    "Since {corporation} built its office in {location}, {product} sales increased.",
    "Because {person} joined {group}, {creative-work} was completed faster.",
    "Whenever {corporation} launches {product}, {group} provides support.",
    "While {group} was performing {creative-work}, {corporation} announced {product}.",
    "During the conference in {location}, {person} criticized {corporation}.",
    "At the same time that {person} directed {creative-work}, {corporation} developed {product}.",
    "By the time {creative-work} was published, {group} had already performed it in {location}.",
    "As soon as {corporation} released {product}, {person} recommended it.",

    # ==== Compound / Complex Sentences ====
    "Although {person} worked at {corporation}, {person} later joined {group} in {location}.",
    "{creative-work}, which was directed by {person}, was released by {corporation} in {location}.",
    "During the festival in {location}, {person} presented {creative-work} produced by {group}.",
    "{product}, originally designed by {person}, is now manufactured by {corporation} in {location}.",
    "{person}, a well-known member of {group}, introduced {product} during the event in {location}.",
    "{creative-work}, praised by critics, was financed by {corporation}.",
    "{corporation}, which competes with {corporation}, acquired {product} from {group}.",
    "After {person} left {corporation}, {group} invited them to {location}.",
    "{group} released {creative-work}, while {corporation} prepared {product} for launch.",
    "Even though {corporation} supported {group}, {person} opposed the partnership.",

    # ==== Conditional Sentences ====
    "If {person} accepts the offer from {corporation}, {product} will be developed in {location}.",
    "Unless {group} supports {creative-work}, {corporation} will cancel the project.",
    "If {corporation} expands to {location}, {group} will gain access to {product}.",
    "Should {person} publish {creative-work}, {corporation} will distribute it worldwide.",
    "If {product} fails in {location}, {corporation} might replace it with {product}.",

    # ==== Questions ====
    "Who founded {corporation} in {location}?",
    "When did {group} release {creative-work}?",
    "Which {product} was introduced by {corporation}?",
    "Why did {person} leave {corporation} for {group}?",
    "How many times has {group} performed {creative-work} in {location}?",
    "Where was {creative-work} filmed by {corporation}?",
    "What role did {person} play in {creative-work} produced by {group}?",

    # ==== Comparative Sentences ====
    "{product} is more popular in {location} than in {location}.",
    "{creative-work} directed by {person} received better reviews than {creative-work}.",
    "{corporation} has more employees in {location} than {corporation}.",
    "{group} is older than {group}.",
    "{person} sold more {product} than {person} in {location}.",

    # ==== Different Tenses ====
    "{person} had been working at {corporation} before joining {group}.",
    "{corporation} will launch {product} in {location} next year.",
    "{group} has been performing {creative-work} in {location} for years.",
    "{person} will introduce {product} after completing {creative-work}.",
    "{creative-work} had already been reviewed by {person} before {corporation} released it.",

    # ==== Descriptive / Appositive ====
    "{person}, the founder of {corporation}, attended the ceremony in {location}.",
    "{corporation}, a global leader, announced {product} in {location}.",
    "{creative-work}, a bestseller, was adapted into a film by {group}.",
    "{product}, known for its design, was improved by {person}.",
    "{group}, famous for {creative-work}, collaborated with {corporation}.",
]


In [ ]:
# Remove the custom TestDataset class as it's causing issues and is not the standard approach.
# The preprocessed data (list of dictionaries) can be directly used with DataLoader and collate_fn.